# Fisheries competition

In this notebook we're going to investigate a range of different architectures for the [Kaggle fisheries competition](https://www.kaggle.com/c/the-nature-conservancy-fisheries-monitoring/).  The video states that vgg.py and ``vgg_ft()`` from utils.py have been updated to include VGG with batch normalization, but this is not the case.  We've instead created a new file [vgg_bn.py](https://github.com/fastai/courses/blob/master/deeplearning1/nbs/vgg16bn.py) and an additional method ``vgg_ft_bn()`` (which is already in utils.py) which we use in this notebook.

## Setup data dirs

First, we need to copy the dataset to a read-write location.

In [1]:
# %cd home/ubuntu/Sansung
# %cd Sansung

!pwd

!ls


/home/ubuntu/Sansung
command.sh	       __MACOSX     test_stg1	   vgg16bn.py
JsonFile	       output	    test_stg1.zip  vgg16bn.pyc
jupyterlog.log	       README.md    train	   vgg16.py
lesson7-Copy1.ipynb    resnet50.py  train.zip	   vgg16.pyc
lesson7.ipynb	       results	    utils.py
lesson7_jin.ipynb      sample	    utils.pyc
lesson7-jungeun.ipynb  test	    valid


In [2]:
# % cd /output/data/fish

# % cd Sansung

!pwd

!ls

/home/ubuntu/Sansung
command.sh	       __MACOSX     test_stg1	   vgg16bn.py
JsonFile	       output	    test_stg1.zip  vgg16bn.pyc
jupyterlog.log	       README.md    train	   vgg16.py
lesson7-Copy1.ipynb    resnet50.py  train.zip	   vgg16.pyc
lesson7.ipynb	       results	    utils.py
lesson7_jin.ipynb      sample	    utils.pyc
lesson7-jungeun.ipynb  test	    valid


In [3]:
# !sudo mkdir -p /output/data/fish
# %cp /data-fisheries/train.zip /output/data/fish/
# !sudo mkdir -p /output/data/fish/test
# %cp /data-fisheries/test_stg1.zip /output/data/fish/test

# %sudo mkdir -p /output/data/fish/models

In [4]:
from zipfile import ZipFile
%cd
%cd Sansung/output/data/fish
zipper = ZipFile('train.zip')
zipper.extractall()
%cd
%cd Sansung/output/data/fish/test
zipper = ZipFile('test_stg1.zip')
zipper.extractall()

/home/ubuntu
/home/ubuntu/Sansung/output/data/fish
/home/ubuntu
/home/ubuntu/Sansung/output/data/fish/test


In [5]:
%mv test_stg1 test

mv: cannot move 'test_stg1' to 'test/test_stg1': Directory not empty


We create the validation and sample sets in the usual way.

In [6]:
# %cd /output/data/fish
# %cd train
# %mkdir ../valid
%cd
%cd Sansung/train

/home/ubuntu
/home/ubuntu/Sansung/train


In [7]:
from glob import glob
import os
import numpy as np

g = glob('*')
print(g)
for d in g: os.mkdir('../valid/'+d)
# 여기는 이미 파일 만들어져서 에러 나도 그냥 넘기기

['DOL', 'NoF', 'LAG', 'YFT', 'SHARK', 'ALB', 'results', 'BET', 'OTHER']


OSError: [Errno 17] File exists: '../valid/DOL'

In [8]:
g = glob('*/*.jpg')
shuf = np.random.permutation(g)

In [9]:
print(shuf.shape)
print(shuf)

(0,)
[]


In [10]:
for i in range(500): os.rename(shuf[i], '../valid/' + shuf[i])
# 끝까지 돌아서 인덱스 초과 에러남(이미 한 코드기 때문)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [11]:
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

mkdir: cannot create directory ‘../sample’: File exists
mkdir: cannot create directory ‘../sample/train’: File exists
mkdir: cannot create directory ‘../sample/valid’: File exists


In [12]:
from shutil import copyfile

g = glob('*')
for d in g: 
    os.mkdir('../sample/train/'+d)
    os.mkdir('../sample/valid/'+d)
    
# File exists 에러는 앞으로 무시해도 됨: 이미 돈 코드기 때문

OSError: [Errno 17] File exists: '../sample/train/DOL'

In [ ]:
g = glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(400): copyfile(shuf[i], '../sample/train/' + shuf[i])

%cd ../valid

g = glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(200): copyfile(shuf[i], '../sample/valid/' + shuf[i])

%cd ..

In [13]:
%mkdir results
%mkdir sample/results
%cd ../..

mkdir: cannot create directory ‘results’: File exists
mkdir: cannot create directory ‘sample/results’: No such file or directory
/home/ubuntu


# Imports and data loading

In [14]:
!sudo pip uninstall -y theano

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [15]:
# !sudo pip install theano==0.9
import theano
theano.__version__

'0.9.0'

In [16]:
from theano.sandbox import cuda

In [17]:
#%cd ubuntu/Sansung
# !ls 

In [18]:
!pwd

/home/ubuntu


In [19]:
%cd /home/ubuntu/Sansung


/home/ubuntu/Sansung


In [20]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using TensorFlow backend.


In [21]:
#path = "/output/data/fish/sample/"
path = "output/data/fish/"
batch_size=64

In [ ]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 3777 images belonging to 8 classes.
Found 537 images belonging to 8 classes.
Found 3777 images belonging to 8 classes.
Found 537 images belonging to 8 classes.
Found 3000 images belonging to 2 classes.


Sometimes it's helpful to have just the filenames, without the path.

In [ ]:
raw_filenames = [f.split('/')[-1] for f in filenames]
raw_test_filenames = [f.split('/')[-1] for f in test_filenames]
raw_val_filenames = [f.split('/')[-1] for f in val_filenames]

## Basic VGG

We start with our usual VGG approach.  We will be using VGG with batch normalization.  We explained how to add batch normalization to VGG in the [imagenet_batchnorm notebook](https://github.com/fastai/courses/blob/master/deeplearning1/nbs/imagenet_batchnorm.ipynb).  VGG with batch normalization is implemented in [vgg_bn.py](https://github.com/fastai/courses/blob/master/deeplearning1/nbs/vgg16bn.py), and there is a version of ``vgg_ft`` (our fine tuning function) with batch norm called ``vgg_ft_bn`` in [utils.py](https://github.com/fastai/courses/blob/master/deeplearning1/nbs/utils.py).

### Initial model

First we create a simple fine-tuned VGG model to be our starting point.

In [ ]:
from vgg16bn import Vgg16BN
model = vgg_ft_bn(8)

In [ ]:
trn = get_data(path+'train')
val = get_data(path+'valid')

In [ ]:
test = get_data(path+'test')

In [ ]:
!pwd

In [ ]:
save_array(path+'results/trn.dat', trn)
save_array(path+'results/val.dat', val)

In [ ]:
save_array(path+'results/test.dat', test)

In [ ]:
trn = load_array(path+'results/trn.dat')
val = load_array(path+'results/val.dat')

In [ ]:
test = load_array(path+'results/test.dat')

In [ ]:
gen = image.ImageDataGenerator()

In [ ]:
model.compile(optimizer=Adam(1e-3),
       loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(trn, trn_labels, batch_size=batch_size, nb_epoch=3, validation_data=(val, val_labels))

In [ ]:
model.save_weights(path+'results/ft1.h5')

### Precompute convolutional output

We pre-compute the output of the last convolution layer of VGG, since we're unlikely to need to fine-tune those layers. (All following analysis will be done on just the pre-computed convolutional features.)

In [ ]:
model.load_weights(path+'results/ft1.h5')

In [ ]:
conv_layers,fc_layers = split_at(model, Convolution2D)

In [ ]:
conv_model = Sequential(conv_layers)

In [ ]:
conv_feat = conv_model.predict(trn)
conv_val_feat = conv_model.predict(val)

In [ ]:
conv_test_feat = conv_model.predict(test)

In [ ]:
save_array(path+'results/conv_val_feat.dat', conv_val_feat)
save_array(path+'results/conv_feat.dat', conv_feat)

In [ ]:
save_array(path+'results/conv_test_feat.dat', conv_test_feat)

In [ ]:
conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')

In [ ]:
conv_test_feat = load_array(path+'results/conv_test_feat.dat')

In [ ]:
conv_val_feat.shape

### Train model

We can now create our first baseline model - a simple 3-layer FC net.

In [ ]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        BatchNormalization(axis=1),
        Dropout(p/4),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(8, activation='softmax')
    ]

In [ ]:
p=0.6

In [ ]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=3, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.optimizer.lr = 1e-4

In [ ]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=7, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.save_weights(path+'models/conv_512_6.h5')

In [ ]:
bn_model.evaluate(conv_val_feat, val_labels)

In [ ]:
bn_model.load_weights(path+'models/conv_512_6.h5')

## Multi-input

The images are of different sizes, which are likely to represent the boat they came from (since different boats will use different cameras). Perhaps this creates some data leakage that we can take advantage of to get a better Kaggle leaderboard position? To find out, first we create arrays of the file sizes for each image:

In [ ]:
sizes = [PIL.Image.open(path+'train/'+f).size for f in filenames]
id2size = list(set(sizes))
size2id = {o:i for i,o in enumerate(id2size)}

In [ ]:
import collections
collections.Counter(sizes)

Then we one-hot encode them (since we want to treat them as categorical) and normalize the data.

In [ ]:
trn_sizes_orig = to_categorical([size2id[o] for o in sizes], len(id2size))

In [ ]:
raw_val_sizes = [PIL.Image.open(path+'valid/'+f).size for f in val_filenames]
val_sizes = to_categorical([size2id[o] for o in raw_val_sizes], len(id2size))

In [ ]:
trn_sizes = trn_sizes_orig-trn_sizes_orig.mean(axis=0)/trn_sizes_orig.std(axis=0)
val_sizes = val_sizes-trn_sizes_orig.mean(axis=0)/trn_sizes_orig.std(axis=0)

To use this additional "meta-data", we create a model with multiple input layers - `sz_inp` will be our input for the size information.

In [ ]:
p=0.6

In [ ]:
inp = Input(conv_layers[-1].output_shape[1:])
sz_inp = Input((len(id2size),))
bn_inp = BatchNormalization()(sz_inp)

x = MaxPooling2D()(inp)
x = BatchNormalization(axis=1)(x)
x = Dropout(p/4)(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p/2)(x)
x = merge([x,bn_inp], 'concat')
x = Dense(8, activation='softmax')(x)

When we compile the model, we have to specify all the input layers in an array.

In [ ]:
model = Model([inp, sz_inp], x)
model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

And when we train the model, we have to provide all the input layers' data in an array.

In [ ]:
model.fit([conv_feat, trn_sizes], trn_labels, batch_size=batch_size, nb_epoch=3, 
             validation_data=([conv_val_feat, val_sizes], val_labels))

In [ ]:
bn_model.optimizer.lr = 1e-4

In [ ]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=8, 
             validation_data=(conv_val_feat, val_labels))

The model did not show an improvement by using the leakage, other than in the early epochs. This is most likely because the information about what boat the picture came from is readily identified from the image itself, so the meta-data turned out not to add any additional information.

## Bounding boxes & multi output

### Import / view bounding boxes

A kaggle user has created bounding box annotations for each fish in each training set image. You can download them [from here](https://www.kaggle.com/c/the-nature-conservancy-fisheries-monitoring/forums/t/25902/complete-bounding-box-annotation). We will see if we can utilize this additional information. First, we'll load in the data, and keep just the largest bounding box for each image.

In [ ]:
#import ujson as json

In [ ]:
anno_classes = ['alb', 'bet', 'dol', 'lag', 'other', 'shark', 'yft']

In [ ]:
!pwd

In [ ]:
def get_annotations():
    annot_urls = {
        '5458/bet_labels.json': 'bd20591439b650f44b36b72a98d3ce27',
        '5459/shark_labels.json': '94b1b3110ca58ff4788fb659eda7da90',
        '5460/dol_labels.json': '91a25d29a29b7e8b8d7a8770355993de',
        '5461/yft_labels.json': '9ef63caad8f076457d48a21986d81ddc',
        '5462/alb_labels.json': '731c74d347748b5272042f0661dad37c',
        '5463/lag_labels.json': '92d75d9218c3333ac31d74125f2b380a'
    }
    cache_subdir = os.path.abspath(os.path.join(path, 'annos'))
    #url_prefix = 'https://kaggle2.blob.core.windows.net/forum-message-attachments/147157/'
    #https://storage.googleapis.com/kaggle-forum-message-attachments/147157/5458/bet_labels.json
    url_prefix = 'https://storage.googleapis.com/kaggle-forum-message-attachments/147157/'
    #http://52.203.141.131:8080/tree/Sansung/JsonFile
    
    if not os.path.exists(cache_subdir):
        os.makedirs(cache_subdir)
    
    for url_suffix, md5_hash in annot_urls.iteritems():
        fname = url_suffix.rsplit('/', 1)[-1]
        #get_file(fname, url_prefix , cache_subdir=cache_subdir, md5_hash=md5_hash)
        get_file(fname, url_prefix+ url_suffix, cache_subdir=cache_subdir, md5_hash=md5_hash)

In [ ]:
get_annotations()

In [ ]:
bb_json = {}
for c in anno_classes:
    if c == 'other': continue # no annotation file for "other" class
    j = json.load(open('{}annos/{}_labels.json'.format(path, c), 'r'))
    for l in j:
        if 'annotations' in l.keys() and len(l['annotations'])>0:
            bb_json[l['filename'].split('/')[-1]] = sorted(
                l['annotations'], key=lambda x: x['height']*x['width'])[-1]

In [ ]:
bb_json['img_04908.jpg']

In [ ]:
file2idx = {o:i for i,o in enumerate(raw_filenames)}
val_file2idx = {o:i for i,o in enumerate(raw_val_filenames)}

For any images that have no annotations, we'll create an empty bounding box.

In [ ]:
empty_bbox = {'height': 0., 'width': 0., 'x': 0., 'y': 0.}

In [ ]:
for f in raw_filenames:
    if not f in bb_json.keys(): bb_json[f] = empty_bbox
for f in raw_val_filenames:
    if not f in bb_json.keys(): bb_json[f] = empty_bbox

Finally, we convert the dictionary into an array, and convert the coordinates to our resized 224x224 images.

In [ ]:
bb_params = ['height', 'width', 'x', 'y']
def convert_bb(bb, size):
    bb = [bb[p] for p in bb_params]
    conv_x = (224. / size[0])
    conv_y = (224. / size[1])
    bb[0] = bb[0]*conv_y
    bb[1] = bb[1]*conv_x
    bb[2] = max(bb[2]*conv_x, 0)
    bb[3] = max(bb[3]*conv_y, 0)
    return bb

In [ ]:
trn_bbox = np.stack([convert_bb(bb_json[f], s) for f,s in zip(raw_filenames, sizes)], 
                   ).astype(np.float32)
val_bbox = np.stack([convert_bb(bb_json[f], s) 
                   for f,s in zip(raw_val_filenames, raw_val_sizes)]).astype(np.float32)

Now we can check our work by drawing one of the annotations.

In [ ]:
def create_rect(bb, color='red'):
    return plt.Rectangle((bb[2], bb[3]), bb[1], bb[0], color=color, fill=False, lw=3)

def show_bb(i):
    bb = val_bbox[i]
    plot(val[i])
    plt.gca().add_patch(create_rect(bb))

In [ ]:
show_bb(0)

### Create & train model

Since we're not allowed (by the kaggle rules) to manually annotate the test set, we'll need to create a model that predicts the locations of the bounding box on each image. To do so, we create a model with multiple outputs: it will predict both the type of fish (the 'class'), and the 4 bounding box coordinates. We prefer this approach to only predicting the bounding box coordinates, since we hope that giving the model more context about what it's looking for will help it with both tasks.

In [ ]:
p=0.6

In [ ]:
inp = Input(conv_layers[-1].output_shape[1:])
x = MaxPooling2D()(inp)
x = BatchNormalization(axis=1)(x)
x = Dropout(p/4)(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p/2)(x)
x_bb = Dense(4, name='bb')(x)
x_class = Dense(8, activation='softmax', name='class')(x)

Since we have multiple outputs, we need to provide them to the model constructor in an array, and we also need to say what loss function to use for each. We also weight the bounding box loss function down by 1000x since the scale of the cross-entropy loss and the MSE is very different.

In [ ]:
model = Model([inp], [x_bb, x_class])
model.compile(Adam(lr=0.001), loss=['mse', 'categorical_crossentropy'], metrics=['accuracy'],
             loss_weights=[.001, 1.])

In [ ]:
model.fit(conv_feat, [trn_bbox, trn_labels], batch_size=batch_size, nb_epoch=3, 
             validation_data=(conv_val_feat, [val_bbox, val_labels]))

In [ ]:
model.optimizer.lr = 1e-5

In [ ]:
model.fit(conv_feat, [trn_bbox, trn_labels], batch_size=batch_size, nb_epoch=10, 
             validation_data=(conv_val_feat, [val_bbox, val_labels]))

Excitingly, it turned out that the classification model is much improved by giving it this additional task. Let's see how well the bounding box model did by taking a look at its output.

In [ ]:
pred = model.predict(conv_val_feat[0:10])

In [ ]:
def show_bb_pred(i):
    bb = val_bbox[i]
    bb_pred = pred[0][i]
    plt.figure(figsize=(6,6))
    plot(val[i])
    ax=plt.gca()
    ax.add_patch(create_rect(bb_pred, 'yellow'))
    ax.add_patch(create_rect(bb))

The image shows that it can find fish that are tricky for us to see!

In [ ]:
show_bb_pred(6)

In [ ]:
model.evaluate(conv_val_feat, [val_bbox, val_labels])

In [ ]:
model.save_weights(path+'models/bn_anno.h5')

In [ ]:
model.load_weights(path+'models/bn_anno.h5')

## Larger size

### Set up data

Let's see if we get better results if we use larger images. We'll use 640x360, since it's the same shape as the most common size we saw earlier (1280x720), without being too big.

In [ ]:
!pwd

In [ ]:
trn = get_data(path+'train', (224,224))
val = get_data(path+'valid', (224,224))

The image shows that things are much clearer at this size.

In [ ]:
plot(trn[0])

In [ ]:
test = get_data(path+'test', (224,224))

In [ ]:
save_array(path+'results/trn_640.dat', trn)
save_array(path+'results/val_640.dat', val)

In [ ]:
save_array(path+'results/test_640.dat', test)

In [ ]:
trn = load_array(path+'results/trn_640.dat')
val = load_array(path+'results/val_640.dat')

We can now create our VGG model - we'll need to tell it we're not using the normal 224x224 images, which also means it won't include the fully connected layers (since they don't make sense for non-default sizes). We will also remove the last max pooling layer, since we don't want to throw away information yet.

In [ ]:
vgg640 = Vgg16BN((224,224)).model
vgg640.pop()
vgg640.input_shape, vgg640.output_shape
vgg640.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

We can now pre-compute the output of the convolutional part of VGG.

In [ ]:
conv_val_feat = vgg640.predict(val, batch_size=32, verbose=1)
conv_trn_feat = vgg640.predict(trn, batch_size=32, verbose=1)

In [ ]:
save_array(path+'results/conv_val_640.dat', conv_val_feat)
save_array(path+'results/conv_trn_640.dat', conv_trn_feat)

In [ ]:
conv_test_feat = vgg640.predict(test, batch_size=32, verbose=1)

In [ ]:
save_array(path+'results/conv_test_640.dat', conv_test_feat)

In [ ]:
conv_val_feat = load_array(path+'results/conv_val_640.dat')
conv_trn_feat = load_array(path+'results/conv_trn_640.dat')

In [ ]:
conv_test_feat = load_array(path+'results/conv_test_640.dat')

## Submit

In [ ]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/7, mx)

In [ ]:
model.evaluate(conv_val_feat, val_labels, batch_size*2)
# help(model)
# dir(model)
model.summary()

In [ ]:
preds = model.predict(conv_test_feat, batch_size=batch_size)

In [ ]:
preds = preds[1]

In [ ]:
test = load_array(path+'results/test_640.dat')

In [ ]:
test = load_array(path+'results/test.dat')

In [ ]:
preds = conv_model.predict(test, batch_size=32)

In [ ]:
subm = do_clip(preds,0.82)

In [ ]:
subm_name = path+'results/subm_bb.gz'

In [ ]:
# classes = sorted(batches.class_indices, key=batches.class_indices.get)
classes = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

In [ ]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'image', raw_test_filenames)
submission.head()

In [ ]:
submission.to_csv(subm_name, index=False, compression='gzip')

In [ ]:
FileLink(subm_name)

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

results=confusion_matrix(conv_val_feat,val_labels)
print(results)

In [ ]:
import seaborn as sn
df_cm = pd.DataFrame(results,range(2),range(2))
plt.figure(figsize=(10,7))
sn.heatmap(df_cm, annot=True)

from sklearn.metrics import classification_report
cfreport=classification_report(conv_val_feat, val_labelst)
print("분류예측 레포트:\n", cfreport)